In [1]:
import torch
import math
import numpy as np 
import cv2

In [2]:
#!python train.py --img 640 --batch-size -1 --epochs 1000 --data passport_rotation7.yaml --weights yolov5m.pt --hyp hyp.passport.rotation7.yaml --cache 'ram' --workers 0

In [9]:
model = torch.hub.load('ultralytics/yolov5','custom',  path='povorot7.pt') #работает, но загружает каждый раз

Using cache found in C:\Users\Алексей/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-8-24 Python-3.8.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 290 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


In [10]:
#надо повернуть так, чтобы обрезки не произошло.
#https://stackoverflow.com/questions/43892506/opencv-python-rotate-image-without-cropping-side
def rotate_image(mat, angle, point):
   

    height, width = mat.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape
    #image_center = point

    rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos = abs(rotation_mat[0,0]) 
    abs_sin = abs(rotation_mat[0,1])

    # find the new width and height bounds
    bound_w = int(height * abs_sin + width * abs_cos)
    bound_h = int(height * abs_cos + width * abs_sin)

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_mat[0, 2] += bound_w/2 - image_center[0]
    rotation_mat[1, 2] += bound_h/2 - image_center[1]

    # rotate image with the new bounds and translated rotation matrix
    rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
    return rotated_mat

In [11]:
def get_crop(file):
    results = model(file)
    pd = results.pandas().xyxy[0]
    pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)
    pd = pd.assign(centre_y = pd.ymin + (pd.ymax-pd.ymin)/2)
    x = pd.centre_x.to_list()
    y = pd.centre_y.to_list()
    #print(pd)
    
    #считаем прижатось к каждой оси, чтобы не считать тан 90 и так получается точнее, нет углов 89.99999999
    delta_x = max(x) - min(x)
    delta_y = max(y) - min(y)
    #print('delta_x  delta_y', delta_x,delta_y)
    if delta_x > delta_y:
        var = 1 #больше прижата к Х
    else: 
        var = 2 #больше прижата к У
    if var == 1:
        koef = np.polyfit(np.array(x),np.array(y),  1)
        degree = math.degrees(math.atan(koef[0]))
    else:
        koef = np.polyfit(np.array(y),np.array(x),  1)
        degree = math.degrees(math.atan(koef[0]))
        #print('Угол во втором варианте',degree)
        if degree > 0:
            degree = 90  - degree
        elif degree == 0: #не убирать
            degree = 0
        else:
            degree =  - (90+ degree)
    #print('Угол по расчетному коэффициенту', degree)         
            
    #получим координаты нижней части там где фото
    #///// может это можно по другому написать, но так работает мгновенно, остальные мои вараинты почему то тупят.
    tmp = pd.loc[pd['name']=='niz']
    for index, row in tmp.iterrows(): 
        N = (row['centre_x'], row['centre_y'])
        break
    #получим координаты верха, там где печать
    tmp = pd.loc[pd['name']=='verh']
    for index, row in tmp.iterrows(): 
        V = (row['centre_x'], row['centre_y'])
        break
    #V и N - это центры верха и низа
    #print('var ',var)
    
    #смотрим на какой угол повернуть. Не много математики
    #if var == 1:
    if N[0] > V[0]:
        print('низ справа')
        degree = -(90 - degree) 
    else:
        print('низ слева')
        degree = (90 + degree)

    img = cv2.imread(file)             #читаем файл
    img = rotate_image(img, degree,N)  #вращаем той процедурой, что выше
    
    #tmp_file = "Y:\\1.jpg" #можно сохранить повернутый, но еще не обрезанный результат
    #cv2.imwrite(tmp_file, img)
    
    results = model(img)
    
    pd = results.pandas().xyxy[0]
    #определяем координаты вырезки
    x1 =int(pd.xmin.min())
    x2 = int(pd.xmax.max())
    
    y1 = int(pd.ymin.min())
    y2 = int(pd.ymax.max())
    
    crop_img = img[y1:y2,x1:x2]
    
    #tmp_file = "Y:\\2.jpg"
    #cv2.imwrite(tmp_file, crop_img)
    #return tmp_file
    #здесь можно вернуть файл или можно вернуть img
    return crop_img

In [ ]:
#берем файл, 
f ='Y:\\test\\22.jpg'
#отправляем его на поворот и кроп.
img = get_crop(f)